In [1]:
# Dependencies and setup
import requests
import matplotlib.pyplot
import pandas as pd
import numpy as np
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime

# Determine city from latitued and longitude
from citipy import citipy

# Get api key from config file, an untracked file in gitignore
from config import api_key

output_files = 'Output/cities.csv'

# Define range of latitudes and longitudes
lat_r=(-45, 45)
lng_r=(-180, 180)

##### Generate List of Cities

In [2]:
# Variable with an empty list to hold cities and lat_long
cities = []
lat_long = []

# Randomize creation of latitude & longitude. Set np size to 1000; change if needed
lat = np.random.uniform(low=-45.00, high=45.00, size=10)
lng = np.random.uniform(low=-180, high = 180, size=10)
lat_long = zip(lat, lng)

# Find nearest city with geo coordinates
for lat_lng in lat_long:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    if city not in cities:
        cities.append(city)

# Validate sufficient number of cities generated        
len(cities)

10

##### API Calls

In [3]:
base_url = 'http://api.openweathermap.org/data/2.5/weather?'
units = 'imperial'

# Setup URL query
query_url = f'{base_url}appid={api_key}&units={units}&q='

# Dictionary of variables where data from api calls will be stored
city_data = {'city_n':[], 'city_lat':[], 'city_lng':[], 'temp':[], 'humdty':[],
             'cloudns':[], 'wndspd':[], 'cntry':[], 'date':[]}

In [4]:
city_ttls = len(cities)
r = 1

print('Retrieving weather data')
print('-' * 30)

for r, city in enumerate(cities):
    try:
        print(f'Retrieving number {r} of {city_ttls}.')
        r+=1
        city = requests.get(query_url + city).json()
        city_data["city_n"].append(city["name"])
        city_data["city_lat"].append(city["coord"]["lat"])
        city_data["city_lng"].append(city["coord"]["lon"])
        city_data["temp"].append(city["main"]["temp"])
        city_data["humdty"].append(city["main"]["humidity"])
        city_data["cloudns"].append(city["clouds"]["all"])
        city_data["wndspd"].append(city["wind"]["speed"])
        city_data["cntry"].append(city["sys"]["country"])
        city_data["date"].append(city["dt"])
        
        # Use timer to delay request to not exceed query limits.
        time.sleep(0.75)
    except:
        print(f'Incomplete record for {city}. Skipping {city}.')
        pass

print(f'Data retrieval completed.')

Retrieving weather data
------------------------------
Retrieving number 0 of 10.
Retrieving number 1 of 10.
Retrieving number 2 of 10.
Retrieving number 3 of 10.
Retrieving number 4 of 10.
Retrieving number 5 of 10.
Retrieving number 6 of 10.
Retrieving number 7 of 10.
Retrieving number 8 of 10.
Retrieving number 9 of 10.
Data retrieval completed.


In [6]:
city_data_df = pd.DataFrame(city_data)
city_data_df.to_csv('Output/city_data.csv')
city_df.head()

,city_n,city_lat,city_lng,temp,humdty,cloudns,wndspd,cntry,date
0,Faya,18.3851,42.4509,64.92,21,42,3.65,SA,1622850647
1,Chachapoyas,-6.2317,-77.8690,55.62,96,89,3.36,PE,1622850648
2,George Town,5.4112,100.3354,78.96,91,20,3.00,MY,1622850471
3,Touros,-5.1989,-35.4608,76.10,80,3,11.48,BR,1622850650
4,Gushikawa,26.3544,127.8686,83.70,84,90,12.82,JP,1622850652
